In [1]:
import waterhealer as wh
import json

In [2]:
source = wh.from_kafka(
    ['testing'],
    {
        'bootstrap.servers': 'localhost:9092',
        'group.id': 'group',
        'auto.offset.reset': 'latest',
    },
    healer = True)

In [3]:
def json_loads(row):
    return (row[0], json.loads(row[1]))

def plus(row):
    no = row[1]['no']
    if no > 10:
        raise Exception('row is bigger than 10')
    return (row[0], no + 1)

In [4]:
outputs = []
source.map(json_loads).map(plus).map(wh.healing, stream = source, callback = print).sink(outputs.append)

Output()

In [5]:
source.start()

In [6]:
from kafka import KafkaProducer
from kafka.partitioner import RoundRobinPartitioner

In [7]:
producer = KafkaProducer(
    bootstrap_servers = ['localhost:9092'],
    api_version = (0, 10),
    partitioner = RoundRobinPartitioner(),
)

In [8]:
producer.bootstrap_connected()

True

In [9]:
from datetime import datetime
import json
from itertools import cycle

def publish_message(producer_instance, topic_name, key, value):
    try:
        key_bytes = bytes(key, encoding = 'utf-8')
        value_bytes = bytes(value, encoding = 'utf-8')
        x = producer_instance.send(topic_name, value = value_bytes)
        return True
    except Exception as ex:
        print('Exception in publishing message')
        print(str(ex))
        return False

## purposely raise the error

Based on function `plus`, if input bigger than 10, it will throw error.

In [13]:
data = {'no': 11, 'datetime': str(datetime.now())}
publish_message(
    producer, 'testing', 'streaming', json.dumps(data)
)

True

ERROR:streamz.core:row is bigger than 10
Traceback (most recent call last):
  File "/usr/local/lib/python3.7/site-packages/streamz/core.py", line 578, in update
    result = self.func(x, *self.args, **self.kwargs)
  File "<ipython-input-3-755692a747c8>", line 7, in plus
    raise Exception('row is bigger than 10')
Exception: row is bigger than 10
tornado.application - ERROR - Exception in callback functools.partial(<bound method IOLoop._discard_future_result of <zmq.eventloop.ioloop.ZMQIOLoop object at 0x10e755050>>, <Future finished exception=Exception('row is bigger than 10')>)
Traceback (most recent call last):
  File "/usr/local/lib/python3.7/site-packages/tornado/ioloop.py", line 743, in _run_callback
    ret = callback()
  File "/usr/local/lib/python3.7/site-packages/tornado/ioloop.py", line 767, in _discard_future_result
    future.result()
  File "/usr/local/lib/python3.7/site-packages/tornado/gen.py", line 748, in run
    yielded = self.gen.send(value)
  File "/Users/huseinzol

## check memory

In [15]:
source.memory

{'f2153b60-46f2-11ea-9111-acde48001122': {'partition': 1,
  'offset': 7,
  'topic': 'testing'}}

As you can see, topic `testing` and partition `1`, to get current consumer offset for a topic and a partition,

```python
consumer.committed
```

## consumer offset

In [24]:
import confluent_kafka as ck

source.consumer.committed([ck.TopicPartition('testing', 1)])[0].offset

-1001

-1001 means, this is a new message for this partition. So kafka does not commit our error, which is, this is what we want.